# ENCODING THE MESSAGE 

In [10]:
import wave


with wave.open('Ex3_sound5.wav', mode='rb') as song:
    frame_bytes = bytearray(list(song.readframes(song.getnframes())))

    secret_message = "Father Christmas does not exist"

    # Add dummy data
    string = secret_message + int((len(frame_bytes) - (len(secret_message) * 8 * 8)) / 8) * '#'

    # Convert text to a bit array
    bits = list(map(int, ''.join([bin(ord(i)).lstrip('0b').rjust(8, '0') for i in string])))

    # Replace LSB of each byte of the audio by one bit from the string
    try:
        for i, bit in enumerate(bits):
            if i < len(frame_bytes):
                frame_bytes[i] = (frame_bytes[i] & 254) | bit
        frame_modified = bytes(frame_bytes)

        # Save the modified frame to a new audio file, secrete_file
        with wave.open('secret_file.wav', mode='wb') as f:
            f.setnchannels(song.getnchannels())
            f.setsampwidth(song.getsampwidth())
            f.setframerate(song.getframerate())
            f.writeframes(frame_modified)
    except Exception as e:
        print(f"An error occurred: {e}")


# DECODING THE MESSAGE

In [11]:

input_file = 'secret_file.wav'

input_wave = wave.open(input_file, mode='rb')
frame_bytes_decode = bytearray(list(input_wave.readframes(input_wave.getnframes())))
input_wave.close()

# Extract the LSBs from the frames
extracted = [frame_byte & 1 for frame_byte in frame_bytes_decode]

# Convert the extracted bits back to a string
string = ''.join(chr(int("".join(map(str, extracted[i:i + 8])), 2)) for i in range(0, len(extracted), 8))

# Find the end of the actual message using a unique delimiter
delimiter = "###"
end_index = string.find(delimiter)

# Remove dummy data
decoded = string[:end_index]

print(f"Successfully decoded: {decoded}")


Successfully decoded: Father Christmas does not exist
